In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as img
import numpy as np
import cv2
import os, shutil
from scipy.spatial import distance
import xml.etree.ElementTree as ET

### Annotation 파일 읽어오기
https://codechacha.com/ko/python-parsing-xml-and-pretty-print/

In [2]:
import os 

path = './empty'
annotation_list = os.listdir(path) 
annotation_list.remove('images')
print(annotation_list)

['annotations.xml']


### 함수 클래스 정의

In [3]:
class Utils:
    def __init__(self):
        pass
    
    # 각 행의 n 번째 열을 기준으로 sort (오름차순)
    # x : nd array
    # axis : 기준 열
    def sort_ndarray_by_col(self, x, axis=0):
        size = x.shape[0]
        col = [x[i][axis] for i in range(size)] # 기준 행의 원소들을 추출
        sorted_col = sorted(col) # 추출된 원소들을 정렬
        rank = [sorted_col.index(x[i][axis]) for i in range(size)] # 정렬된 index 추출
        #print(rank)
        try:
            sorted_array = [x[rank.index(i)] for i in range(size)]  # 정렬된 index 순서대로 정렬
        except: # 똑같은 순위가 있는 경우 예외처리
            overlap_rank = []
            for i in range(size):
                if rank.count(i) > 1:
                    overlap_rank.append(i)
            for r in overlap_rank:
                overlap_list = [i for i in range(size) if rank[i]==r]
                for k, idx in enumerate(overlap_list):
                    rank[idx] += k
            sorted_array = [x[rank.index(i)] for i in range(size)]  # 정렬된 index 순서대로 정렬

        return np.array(sorted_array)

    # 함수화
    def perspective_transform(self, src, point, height=256, width=256):    
        # 4 꼭지점 분류 
        sorted_by_y= self.sort_ndarray_by_col(point, 1)
        top_two = sorted_by_y[0:2]
        topLeft, topRight = self.sort_ndarray_by_col(top_two, 0)
        bottom_two = sorted_by_y[2:4]
        bottomLeft, bottomRight = self.sort_ndarray_by_col(bottom_two, 0)
        #print("tl:", topLeft)
        #print("bl", bottomLeft)
        #print("tr", topRight)
        #print("br", bottomRight)


        # 변환 전 4개 좌표 
        srcPoint = np.float32([topLeft, topRight, bottomRight , bottomLeft])

        # 변환 후 4개 좌표
        dstPoint = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)

        # Perspective transformation
        matrix = cv2.getPerspectiveTransform(srcPoint, dstPoint)
        dst = cv2.warpPerspective(src, matrix, (width, height))

        return dst

    # 이미지 상에 polygon 그리기
    def draw_label(self, img_path, points, save_path=""):
        '''
        image = img.imread(img_path)
        plt.imshow(image)

        ax = plt.gca()

        point = np.array([[data['p1'][0], data['p1'][1]],
                          [data['p2'][0], data['p2'][1]],
                          [data['p3'][0], data['p3'][1]],
                          [data['p4'][0], data['p4'][1]]])
        #print(point)

        polygon = patches.Polygon(point, fill=None ,edgecolor='k',ls='solid',lw=3)

        ax.add_patch(polygon)

        plt.show()
        '''
        src = cv2.imread(img_path, cv2.IMREAD_COLOR)
        dst = cv2.polylines(src, np.int32([points]), True, (255, 0, 0), 6)

        return dst

    def extract_bb_points(self, points, width, height):
        if len(points.split(';')) == 4: # 라벨링된 point가 4개인 경우 = polygon 라벨링 (없음)
            ps = [(float(points.split(';')[0].split(',')[0]), float(points.split(';')[0].split(',')[1])),
                 (float(points.split(';')[1].split(',')[0]), float(points.split(';')[1].split(',')[1])),
                 (float(points.split(';')[2].split(',')[0]), float(points.split(';')[2].split(',')[1])),
                 (float(points.split(';')[3].split(',')[0]), float(points.split(';')[3].split(',')[1]))]

            # 4 꼭지점 분류 
            sorted_by_y= self.sort_ndarray_by_col(np.array(ps), 1)
            top_two = sorted_by_y[0:2]
            topLeft, topRight = self.sort_ndarray_by_col(top_two, 0)
            bottom_two = sorted_by_y[2:4]
            bottomLeft, bottomRight = list(self.sort_ndarray_by_col(bottom_two, 0))

        elif len(points.split(';')) == 2: # 라벨링된 point가 2개인 경우 = 직선 라벨링 (있음))
            ps = [(float(points.split(';')[0].split(',')[0]), float(points.split(';')[0].split(',')[1])),
                 (float(points.split(';')[1].split(',')[0]), float(points.split(';')[1].split(',')[1]))]

            # 2 꼭지점 분류
            if ps[0][0] < ps[1][0]:
                bottomLeft = ps[0]
                bottomRight = ps[1]
            else:
                bottomLeft = ps[1]
                bottomRight = ps[0]
        else:
            return None, 0, None

        print(len(points.split(';')))
        print('bb', bottomLeft, bottomRight) # bounding box 생성에 기준이 되는 주차면 앞 라인의 두 점

        # 라인으로부터 바운딩 박스 생성
        x_offset = 0
        width = distance.euclidean(bottomLeft, bottomRight)+x_offset
        y_offset = width/10.0
        height = -width*2/3.0
        #print(ps, width, height)


        b1 = (bottomLeft[0], bottomLeft[1]+y_offset)
        b2 = (bottomRight[0], bottomRight[1]+y_offset)
        b3 = (b2[0], b2[1]+height)
        b4 = (b1[0], b1[1]+height)
        print('ps:',np.array([b1, b2, b3, b4]), width, np.array(ps))

        return np.array([b1, b2, b3, b4]), width, np.array(ps)

    def get_occupied_info(self, polyline, width, height):
        label = polyline.get('label')
        points = polyline.get('points') # 2개
        bb_points, width, _ = self.extract_bb_points(points, width, height)
        return label, width, bb_points

    def get_empty_info(self, polygon, width, height):
        label = polygon.get('label')
        points = polygon.get('points') # 4개 or 5개
        bb_points, width, poly_points = self.extract_bb_points(points, width, height)
        empty_type_dict = {'일반형':0, '경형':1, '장애인전용':2, '여성우선주차장':3, '환경친화적 자동차 전용':4, 'etc':5, '기타':5}
        empty_type = empty_type_dict[polygon.find('attribute').text]
        return label, width, bb_points, poly_points, empty_type

    def get_obstacle_info(self, polygon, width, height):
        label = polygon.get('label')
        points = polygon.get('points')
        bb_points, width, poly_points = self.extract_bb_points(points, width, height)
        obstacle_type_dict = {'사람':0, '이륜차':1, '기타':2}
        obs_type = obstacle_type_dict[polygon.find('attribute').text]
        return label, width, bb_points, poly_points, obs_type

In [4]:
#from utils import Utils
# 함수 클래스 인스턴스화
ut = Utils()

### 데이터셋 확인

In [24]:
for annotation in annotation_list:
    # annotation 파일 불러오기
    filePath = os.path.join(path, annotation)
    #print(filePath)
    tree = ET.parse(filePath)
    root = tree.getroot()
    dataset_name = root.find("meta").find("task").find("name").text
    print(annotation, dataset_name)

annotations.xml Extend_Empty


### xml 파일로부터 라벨링 정보 추출

In [25]:
occupied_cnt, empty_cnt, obstacle_cnt = 0, 0, 0

datasets = {}
datasets['occupied'] = {}
datasets['empty'] = {}
datasets['obstacle'] = {}

for annotation in annotation_list:
    print(annotation)
    # annotation 파일 불러오기
    filePath = os.path.join(path, annotation)
    #print(filePath)
    tree = ET.parse(filePath)
    root = tree.getroot()
    dataset_name = root.find("meta").find("task").find("name").text
    
    for image in root.findall("image"):
        image_attrib = image.attrib
        image_frame = image_attrib.get('id')
        image_name = image_attrib.get('name')
        print(image_name)
        
        if not os.path.exists(os.path.join(path, 'images', image_name)): # 이미지가 존재하지 않을 경우 pass
            continue
        
        image_witdh = image_attrib.get('width')
        image_height = image_attrib.get('height')
        polylines = image.findall('polyline') # 있음 라벨링
        polygons = image.findall('polygon') # 없음 라벨링
        
        for polyline in polylines: # 있음
            polyline_label, width, bb_points = ut.get_occupied_info(polyline, image_witdh, image_height)
            print(polyline_label) 
            if width > 250: # 길이가 너무 짧은 경우 넘어감
                occupied_cnt += 1 
                datasets['occupied'][str(occupied_cnt)]={'label':polyline_label, 'src':image_name,
                                                         'dataset':dataset_name,
                                                         'frame':image_frame,
                                                         'width': width,
                                                         'bb_points':bb_points
                                                        }
            
        for polygon in polygons:
            label = polygon.get('label')
            if label == '없음': # 없음
                empty_label, width, bb_points, poly_points, empty_type = ut.get_empty_info(polygon, image_witdh, image_height)
                print(empty_label, empty_type)
                if (type(bb_points) != None) and (width > 250): # point 정보가 없거나 길이가 너무 짧은 경우 넘어감
                    empty_cnt += 1
                    datasets['empty'][str(empty_cnt)]={'label':empty_label, 'src':image_name,
                                                       'dataset':dataset_name,
                                                       'frame':image_frame,
                                                       'width':width,
                                                       'type':empty_type,
                                                       'bb_points':bb_points,
                                                       'poly_points':poly_points
                                                      }
                
            elif label == '장애물': # 장애물
                obs_label, width, bb_points, poly_points, obs_type = ut.get_obstacle_info(polygon, image_witdh, image_height)
                print(obs_label, obs_type)
                obstacle_cnt += 1
                datasets['obstacle'][str(obstacle_cnt)]={'label':obs_label, 'src':image_name,
                                                         'dataset':dataset_name,
                                                         'frame':image_frame,
                                                         'width': width,
                                                         'type':obs_type,
                                                         'poly_points':poly_points
                                                        }
                
        #occupied_cnt += len(polylines)
        #empty_cnt += len(polygons)
        #print(image_name, '| 있음', len(polylines), '없음', len(polygons))
    

annotations.xml
extend1.jpg
4
bb [ 167.35 1809.4 ] [1361.8 1802. ]
ps: [[ 167.35       1928.84729225]
 [1361.8        1921.44729225]
 [1361.8        1125.1320106 ]
 [ 167.35       1132.5320106 ]] 1194.4729224641303 [[1030.6  1247.54]
 [ 167.35 1809.4 ]
 [1361.8  1802.  ]
 [1559.   1232.7 ]]
없음 1
extend10.jpg
4
bb [ 169.35 1820.21] [1363.8  1812.81]
ps: [[ 169.35       1939.65729225]
 [1363.8        1932.25729225]
 [1363.8        1135.9420106 ]
 [ 169.35       1143.3420106 ]] 1194.4729224641303 [[1032.61 1258.35]
 [ 169.35 1820.21]
 [1363.8  1812.81]
 [1561.   1243.51]]
없음 1
extend100.jpg
4
bb [1753.93 1603.11] [2530.22 1626.81]
ps: [[1753.93       1680.77516942]
 [2530.22       1704.47516942]
 [2530.22       1186.70737329]
 [1753.93       1163.00737329]] 776.6516941975982 [[1839.85 1203.11]
 [1753.93 1603.11]
 [2530.22 1626.81]
 [2198.37 1217.93]]
없음 1
4
bb [2565.78 1626.81] [3339.11 1656.44]
ps: [[2565.78       1704.19974259]
 [3339.11       1733.82974259]
 [3339.11       1217.8981253

4
bb [1214.44 1833.09] [2108.93 1524.62]
ps: [[1214.44       1927.70850247]
 [2108.93       1619.23850247]
 [2108.93        988.44848599]
 [1214.44       1296.91848599]] 946.1850247176816 [[ 939.42 1334.46]
 [1214.44 1833.09]
 [2108.93 1524.62]
 [1479.18 1262.49]]
없음 5
extend138.jpg
4
bb [1968.85 1599.24] [2698.81 1578.68]
ps: [[1968.85       1672.26494883]
 [2698.81       1651.70494883]
 [2698.81       1164.87195662]
 [1968.85       1185.43195662]] 730.2494883257366 [[1840.34 1216.27]
 [1968.85 1599.24]
 [2698.81 1578.68]
 [2228.45 1231.69]]
없음 1
extend1380.jpg
4
bb [2344.11 1704.63] [3040.66 1537.56]
ps: [[2344.11       1776.26060012]
 [3040.66       1609.19060012]
 [3040.66       1131.65326597]
 [2344.11       1298.72326597]] 716.3060012313172 [[1840.3  1275.4 ]
 [2344.11 1704.63]
 [3040.66 1537.56]
 [2205.32 1211.13]]
없음 5
extend1381.jpg
4
bb [2337.69 1699.44] [3034.24 1532.37]
ps: [[2337.69       1771.07060012]
 [3034.24       1604.00060012]
 [3034.24       1126.46326597]
 [2337.6

4
bb [2995.88 1654.23] [3604.47 1583.56]
ps: [[2995.88       1715.49793917]
 [3604.47       1644.82793917]
 [3604.47       1236.37501138]
 [2995.88       1307.04501138]] 612.6793916886708 [[2351.95 1289.08]
 [2995.88 1654.23]
 [3604.47 1583.56]
 [2726.92 1287.12]]
없음 1
4
bb [2000.54 1760.25] [2799.56 1671.9 ]
ps: [[2000.54       1840.63897206]
 [2799.56       1752.28897206]
 [2799.56       1216.36249166]
 [2000.54       1304.71249166]] 803.8897206084924 [[1778.7  1320.49]
 [2000.54 1760.25]
 [2799.56 1671.9 ]
 [2216.49 1300.86]]
없음 1
4
bb [ 394.65 1923.19] [1515.63 1807.36]
ps: [[ 394.65       2035.88484235]
 [1515.63       1920.05484235]
 [1515.63       1168.75589333]
 [ 394.65       1284.58589333]] 1126.948423531441 [[1089.6  1346.  ]
 [ 394.65 1923.19]
 [1515.63 1807.36]
 [1568.64 1336.2 ]]
없음 1
extend1722.jpg
4
bb [ 394.65 1923.19] [1515.63 1807.36]
ps: [[ 394.65       2035.88484235]
 [1515.63       1920.05484235]
 [1515.63       1168.75589333]
 [ 394.65       1284.58589333]] 1126.

4
bb [1097.48 1641.55] [2069.13 1660.19]
ps: [[1097.48       1738.73287772]
 [2069.13       1757.37287772]
 [2069.13       1109.48702628]
 [1097.48       1090.84702628]] 971.8287771516134 [[1521.55 1280.39]
 [1097.48 1641.55]
 [2069.13 1660.19]
 [1945.63 1282.72]]
없음 3
4
bb [  67.57 1650.87] [1060.19 1655.53]
ps: [[  67.57       1750.13309385]
 [1060.19       1754.79309385]
 [1060.19       1093.03913487]
 [  67.57       1088.37913487]] 992.6309384660545 [[1018.25 1271.07]
 [  67.57 1650.87]
 [1060.19 1655.53]
 [1502.91 1275.73]]
없음 3
4
bb [2087.8 1662.5] [2994.17 1655.53]
ps: [[2087.8        1753.13967993]
 [2994.17       1746.16967993]
 [2994.17       1141.90514706]
 [2087.8        1148.87514706]] 906.396799310324 [[1966.6  1273.4 ]
 [2087.8  1662.5 ]
 [2994.17 1655.53]
 [2383.69 1278.06]]
없음 3
extend2191.jpg
4
bb [2087.8 1662.5] [2994.17 1655.53]
ps: [[2087.8        1753.13967993]
 [2994.17       1746.16967993]
 [2994.17       1141.90514706]
 [2087.8        1148.87514706]] 906.396799

4
bb [1132.36 1541.93] [2079.02 1541.93]
ps: [[1132.36       1636.596     ]
 [2079.02       1636.596     ]
 [2079.02       1005.48933333]
 [1132.36       1005.48933333]] 946.6600000000001 [[1973.05 1126.88]
 [1545.64 1123.35]
 [1132.36 1541.93]
 [2079.02 1541.93]]
없음 3
extend2540.jpg
없음 4
extend2541.jpg
없음 4
extend2542.jpg
없음 4
extend2543.jpg
4
bb [   0.  1640.8] [1437.9 1845.7]
ps: [[   0.         1786.04257021]
 [1437.9        1990.94257021]
 [1437.9        1022.65876883]
 [   0.          817.75876883]] 1452.4257020584564 [[1713.42 1126.88]
 [1079.38 1089.8 ]
 [   0.   1640.8 ]
 [1437.9  1845.7 ]]
없음 4
extend2544.jpg
없음 4
extend2545.jpg
4
bb [   0.  1640.8] [1437.9 1845.7]
ps: [[   0.         1786.04257021]
 [1437.9        1990.94257021]
 [1437.9        1022.65876883]
 [   0.          817.75876883]] 1452.4257020584564 [[1713.42 1126.88]
 [1079.38 1089.8 ]
 [   0.   1640.8 ]
 [1437.9  1845.7 ]]
없음 4
extend2546.jpg
4
bb [   0.  1640.8] [1437.9 1845.7]
ps: [[   0.         1786.04257021]

4
bb [1222.26 1041.14] [1663.79  934.37]
ps: [[1222.26       1086.56560663]
 [1663.79        979.79560663]
 [1663.79        676.95822908]
 [1222.26        783.72822908]] 454.2560663326358 [[1036.81  685.26]
 [1216.96  680.05]
 [1663.79  934.37]
 [1222.26 1041.14]]
없음 5
4
bb [ 626.18 1008.46] [1222.26 1041.14]
ps: [[ 626.18       1068.1575166 ]
 [1222.26       1100.8375166 ]
 [1222.26        702.85407261]
 [ 626.18        670.17407261]] 596.9751659826395 [[ 863.73  677.31]
 [1036.81  685.26]
 [1222.26 1041.14]
 [ 626.18 1008.46]]
없음 5
4
bb [199.66 866.29] [476.94 971.38]
ps: [[ 199.66        895.94267383]
 [ 476.94       1001.03267383]
 [ 476.94        803.34818166]
 [ 199.66        698.25818166]] 296.5267382547483 [[476.94 971.38]
 [199.66 866.29]
 [657.97 670.25]
 [799.26 679.08]]
없음 5
extend2801.jpg
4
bb [1222.26 1041.14] [1663.79  934.37]
ps: [[1222.26       1086.56560663]
 [1663.79        979.79560663]
 [1663.79        676.95822908]
 [1222.26        783.72822908]] 454.2560663326358

4
bb [673.87 819.49] [1075.67  852.16]
ps: [[ 673.87        859.80259963]
 [1075.67        892.47259963]
 [1075.67        623.72193542]
 [ 673.87        591.05193542]] 403.1259963088464 [[1047.41  623.44]
 [ 889.34  626.09]
 [ 673.87  819.49]
 [1075.67  852.16]]
없음 2
extend3121.jpg
4
bb [673.87 819.49] [1075.67  852.16]
ps: [[ 673.87        859.80259963]
 [1075.67        892.47259963]
 [1075.67        623.72193542]
 [ 673.87        591.05193542]] 403.1259963088464 [[1047.41  623.44]
 [ 889.34  626.09]
 [ 673.87  819.49]
 [1075.67  852.16]]
없음 2
extend3122.jpg
4
bb [673.87 819.49] [1075.67  852.16]
ps: [[ 673.87        859.80259963]
 [1075.67        892.47259963]
 [1075.67        623.72193542]
 [ 673.87        591.05193542]] 403.1259963088464 [[1047.41  623.44]
 [ 889.34  626.09]
 [ 673.87  819.49]
 [1075.67  852.16]]
없음 2
extend3123.jpg
4
bb [673.87 819.49] [1075.67  852.16]
ps: [[ 673.87        859.80259963]
 [1075.67        892.47259963]
 [1075.67        623.72193542]
 [ 673.87      

4
bb [ 635.01 1919.  ] [1739.91 1820.98]
ps: [[ 635.01       2029.92393477]
 [1739.91       1931.90393477]
 [1739.91       1192.41103633]
 [ 635.01       1290.43103633]] 1109.2393476612701 [[1739.91 1820.98]
 [ 635.01 1919.  ]
 [1194.88 1373.26]
 [1702.83 1352.95]]
없음 1
4
bb [1739.91 1820.98] [2603.56 1745.04]
ps: [[1739.91       1907.67822409]
 [2603.56       1831.73822409]
 [2603.56       1253.75006347]
 [1739.91       1329.69006347]] 866.9822409369178 [[1702.83 1352.95]
 [2135.53 1329.99]
 [2603.56 1745.04]
 [1739.91 1820.98]]
없음 1
extend53.jpg
4
bb [1639.24 1563.12] [2386.33 1563.12]
ps: [[1639.24  1637.829]
 [2386.33  1637.829]
 [2386.33  1139.769]
 [1639.24  1139.769]] 747.0899999999999 [[1663.97 1227.55]
 [2071.95 1232.85]
 [2386.33 1563.12]
 [1639.24 1563.12]]
없음 1
extend530.jpg
4
bb [2266.9 1798.9] [3088.19 1747.69]
ps: [[2266.9        1881.1885003 ]
 [3088.19       1829.9785003 ]
 [3088.19       1281.38849829]
 [2266.9        1332.59849829]] 822.8850030229011 [[1975.52 1343.2

4
bb [ 120.36 1810.38] [1347.83 1794.49]
ps: [[ 120.36       1933.13728463]
 [1347.83       1917.24728463]
 [1347.83       1098.86538709]
 [ 120.36       1114.75538709]] 1227.5728463109633 [[1042.29 1241.68]
 [1547.4  1259.35]
 [1347.83 1794.49]
 [ 120.36 1810.38]]
없음 1
extend90.jpg
4
bb [ 829.29 1657.61] [1793.61 1625.82]
ps: [[ 829.29       1754.0943856 ]
 [1793.61       1722.3043856 ]
 [1793.61       1079.07514826]
 [ 829.29       1110.86514826]] 964.843856020237 [[1293.   1222.2 ]
 [1738.86 1242.57]
 [1793.61 1625.82]
 [ 829.29 1657.61]]
없음 1
4
bb [  26.69 1543.61] [ 925.66 1571.87]
ps: [[  26.69       1633.55140807]
 [ 925.66       1661.81140807]
 [ 925.66       1062.20202096]
 [  26.69       1033.94202096]] 899.4140806658521 [[ 925.66 1571.87]
 [  26.69 1543.61]
 [ 833.82 1208.05]
 [1293.   1222.2 ]]
없음 1
extend900.jpg
4
bb [2997.4 1716.8] [3840.  1725.6]
ps: [[2997.4        1801.06459517]
 [3840.         1809.86459517]
 [3840.         1248.10062734]
 [2997.4        1239.30062734

In [26]:
datasets

{'occupied': {},
 'empty': {'1': {'label': '없음',
   'src': 'extend1.jpg',
   'dataset': 'Extend_Empty',
   'frame': '0',
   'width': 1194.4729224641303,
   'type': 1,
   'bb_points': array([[ 167.35      , 1928.84729225],
          [1361.8       , 1921.44729225],
          [1361.8       , 1125.1320106 ],
          [ 167.35      , 1132.5320106 ]]),
   'poly_points': array([[1030.6 , 1247.54],
          [ 167.35, 1809.4 ],
          [1361.8 , 1802.  ],
          [1559.  , 1232.7 ]])},
  '2': {'label': '없음',
   'src': 'extend10.jpg',
   'dataset': 'Extend_Empty',
   'frame': '1',
   'width': 1194.4729224641303,
   'type': 1,
   'bb_points': array([[ 169.35      , 1939.65729225],
          [1363.8       , 1932.25729225],
          [1363.8       , 1135.9420106 ],
          [ 169.35      , 1143.3420106 ]]),
   'poly_points': array([[1032.61, 1258.35],
          [ 169.35, 1820.21],
          [1363.8 , 1812.81],
          [1561.  , 1243.51]])},
  '3': {'label': '없음',
   'src': 'extend100.jpg',

### 카테고리별 개수 확인

In [27]:
len(datasets['occupied']), len(datasets['empty']), len(datasets['obstacle'])

(0, 4568, 0)

In [28]:
len(datasets['occupied']) + len(datasets['empty'])

4568

### Patch를 저장할 폴더 및 라벨링 텍스트 파일 생성

In [34]:
dataset_dir = 'empty'

FOLDER_NAME = dataset_dir + '_generated' # [dataset_dir]_generated
DATASET_ROOT = "/root/share/datasets/Bluecom/"
DATASET_PATH = DATASET_ROOT + FOLDER_NAME # /root/share/datasets/[dataset_dir]_generated
EMPTY_PATH = os.path.join(DATASET_PATH, 'Empty') # /root/share/datasets/[dataset_dir]_generated/Empty
#OCCUPIED_PATH = os.path.join(DATASET_PATH, 'Occupied') # /root/share/datasets/[dataset_dir]_generated/Occupied
EMPTY_TYPE_PATH = os.path.join(DATASET_PATH, 'EmptyType') # /root/share/datasets/[dataset_dir]_generated/EmptyType
#OBSTACLE_PATH = os.path.join(DATASET_PATH, 'Obstacle') # /root/share/datasets/[dataset_dir]_generated/Obstacle
LABEL_PATH = os.path.join(DATASET_PATH, 'Label') # /root/share/datasets/[dataset_dir]_generated/Label

# 폴더가 없는 경우 생성
if not os.path.isdir(DATASET_PATH):
    os.makedirs(DATASET_PATH, exist_ok=True)
if not os.path.isdir(EMPTY_PATH):
    os.makedirs(EMPTY_PATH, exist_ok=True)
#if not os.path.isdir(OCCUPIED_PATH):
#    os.makedirs(OCCUPIED_PATH, exist_ok=True)
if not os.path.isdir(EMPTY_TYPE_PATH):
    os.makedirs(EMPTY_TYPE_PATH, exist_ok=True)
#if not os.path.isdir(OBSTACLE_PATH):
#    os.makedirs(OBSTACLE_PATH, exist_ok=True)
if not os.path.isdir(LABEL_PATH):
    os.makedirs(LABEL_PATH, exist_ok=True)
    
# Open label txt file 
label_file_all = open(DATASET_PATH + '/'+ dataset_dir + '_all.txt', 'w')
#label_file_occupied = open(DATASET_PATH + '/' + dataset_dir + '_occupied.txt', 'w')
label_file_empty = open(DATASET_PATH + '/' + dataset_dir + '_empty.txt', 'w')
label_file_empty_type = open(DATASET_PATH + '/' + dataset_dir + '_empty_type.txt', 'w')
#label_file_obstacle = open(DATASET_PATH + '/' + dataset_dir + '_obstacle.txt', 'w')
#label_file_total = open(DATASET_PATH + '/' + dataset_dir + '_total.txt', 'w')

### 이미지 저장 경로 생성 함수

In [35]:
def define_img_path(dataset_dir, data, key, verbose=False):
    img_name = data['src'] 
    # 'DJI_0004_004.MP4_snapshot_00.28_[2022.08.22_12.02.15].jpg'
    img_path = os.path.join(dataset_dir, 'images', img_name) 
    # 'total_20220724_20220818_update/images/DJI_0004_004.MP4_snapshot_00.28_[2022.08.22_12.02.15].jpg'
    data_name = category + key + '_' + img_name.split('.')[0] 
    # occupied1000_DJI_0004_004.MP4_snapshot_00.28_[2022.08.22_12.02.15]
    label_img_name = data_name+'_label.jpg' 
    # occupied1000_DJI_0004_004.MP4_snapshot_00.28_[2022.08.22_12.02.15]_label.jpg
    label_img_path = os.path.join(LABEL_PATH, label_img_name) 
    # /root/share/datasets/[dataset_dir]_generated/Label/occupied1000_DJI_0004_004.MP4_snapshot_00.28_[2022.08.22_12.02.15]_label.jpg
    patch_name = data_name+'.jpg' 
    # occupied1000_DJI_0004_004.MP4_snapshot_00.28_[2022.08.22_12.02.15].jpg
    patch_path = os.path.join(PATCH_PATH, patch_name) 
    # PATCH_PATH/occupied1000_DJI_0004_004.MP4_snapshot_00.28_[2022.08.22_12.02.15].jpg

    if verbose:
        print(img_path)
        print('image size: %d bytes'%os.path.getsize(img_path))
        print(label_img_name)
        print(label_img_path)
        print(patch_name)
        print(patch_path)

    if data['label'] == '없음':
        empty_type_img_name = 'Empty_type_'+str(data['type'])+'_'+patch_name
        empty_type_img_path = os.path.join(EMPTY_TYPE_PATH, empty_type_img_name)
        return img_path, label_img_path, patch_path, empty_type_img_path
    elif data['label'] == '장애물':
        obstacle_img_name = 'Obstacle_type_'+str(data['type'])+'_'+patch_name
        obstacle_img_path = os.path.join(OBSTACLE_PATH, obstacle_img_name)
        return img_path, label_img_path, obstacle_img_path
    else: # 있음
        return img_path, label_img_path, patch_path

### Generate empty patches

In [36]:
cnt = 0
category = 'empty'
PATCH_PATH = EMPTY_PATH

# Open label txt file 
label_file_all_lines = list(open(DATASET_PATH + '/'+ dataset_dir + '_all.txt', 'r').readlines()) if os.path.exists(DATASET_PATH + '/'+ dataset_name + '_all.txt') else []
label_file_empty_lines = list(open(DATASET_PATH + '/' + dataset_dir + '_occupied.txt', 'r').readlines()) if os.path.exists(DATASET_PATH + '/'+ dataset_name + '_occupied.txt') else []
label_file_empty_type_lines = list(open(DATASET_PATH + '/' + dataset_dir + '_empty_type.txt', 'r').readlines()) if os.path.exists(DATASET_PATH + '/'+ dataset_name + '_empty_type.txt') else []

for key in datasets[category].keys():
    print(key)
    dataset = datasets[category]
    data = dataset[key] # label src b1 b2 b3 b4
    bb_points = data['bb_points'] 
    poly_points = data['poly_points']
    
    img_path, label_img_path, patch_path, empty_type_img_path = define_img_path(dataset_dir, data, key)
    
    # check existing file
    if os.path.exists(label_img_path):
        print('label image {} exists'.format(label_img_path.split('/')[-1]))
    if os.path.exists(patch_path):
        print('patch image {} exists'.format(patch_path.split('/')[-1]))
    if os.path.exists(empty_type_img_path):
        print('empty type image {} exists'.format(empty_type_img_path.split('/')[-1]))
        
    # label 이미지, patch, empty type 이미지가 없는 경우에만 생성
    if not os.path.exists(label_img_path) or not os.path.exists(patch_path) or not os.path.exists(empty_type_img_path): 
        # source image
        src = cv2.imread(img_path, cv2.IMREAD_COLOR)

        # labeled image
        if not os.path.exists(label_img_path):
            label_img = ut.draw_label(img_path, bb_points)
            cv2.imwrite(label_img_path, label_img)

        # cropped patch
        if not os.path.exists(patch_path):
            patch_img = ut.perspective_transform(src, bb_points, 256, 256)
            cv2.imwrite(patch_path, patch_img)
        
        # empty type image
        if not os.path.exists(empty_type_img_path): 
            empty_type_img = ut.perspective_transform(src, poly_points, 256, 256)
            cv2.imwrite(empty_type_img_path, empty_type_img)
    
    label_file_all_lines.append(patch_path+' 0\n')
    label_file_empty_lines.append(patch_path+' 0\n')
    label_file_empty_type_lines.append(empty_type_img_path+' '+str(data['type'])+'\n')
        
    cnt += 1
    #if cnt > 50:
    #    break


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702


In [37]:
# 라벨링 텍스트 파일 생성 및 저장
# all (empty + occupied)
with open(DATASET_PATH + '/'+ dataset_dir + '_all.txt','w') as label_file_all:
    for line in set(label_file_all_lines) :
        label_file_all.write(line)
# only empty
with open(DATASET_PATH + '/' + dataset_dir + '_empty.txt','w') as label_file_empty:   
    for line in set(label_file_empty_lines) :
        label_file_empty.write(line)
# only empty type
with open(DATASET_PATH + '/' + dataset_dir + '_empty_type.txt','w') as label_file_empty_type:   
    for line in set(label_file_empty_type_lines) :
        label_file_empty_type.write(line)